In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import ROOT

from root_numpy import root2array,fill_profile
from scipy.stats import norm as StatsNorm    # for gaussian fitting
from ROOT import TProfile

In [ ]:
#Predefine the figure size and font size
%matplotlib inline
plt.rcParams['figure.figsize'] = [10.7, 6]
plt.rcParams['font.size']=20
plt.rcParams['axes.labelsize']=16
plt.rcParams['axes.titlesize']=20
plt.rcParams['figure.dpi']=80
plt.rcParams['legend.fontsize']=16

# 0. Data File and Quantities

In [ ]:
directory = '/scratch/group/mitchcomp/CDMS/data/nolan/Lei_Samples/CDMSlite_Unifield/samples'
# Particle Hits and DMC outputs (combined)
SuperSim_dir = directory+'/hits/custom/*Sep25*/output'
SuperSimfile = sorted(glob.glob(SuperSim_dir+'/*.root'))
# Raw data (combined)
#RawData_dir = directory + '/raw/custom/*may15*/combined_Binary_files'
#RawDatafile = sorted(glob.glob(RawData_dir+'/*.gz'))
# RQ and RRQ
#RQRRQ_dir = directory + '/processed/custom/*may15*/output/submerged/all/bg'
#RQfile    = sorted(glob.glob(RQRRQ_dir+'/merge*.root'))
#RRQfile   = sorted(glob.glob(RQRRQ_dir+'/calib*.root'))

Geometry0 = root2array(SuperSimfile[0], treename="G4SettingsInfoDir/Geometry")
Geometry0 = Geometry0[0]

In [ ]:
#Versions
VersionBranches = root2array(SuperSimfile, treename='G4SettingsInfoDir/Versions')
Versions = pd.DataFrame(VersionBranches)
Versions.iloc[0]

In [ ]:
#SuperSim Macro
f = ROOT.TFile(SuperSimfile[0],'READ')
SuperSim_Macro = f.Get('G4SettingsInfoDir/SuperSim_Macro')
SuperSim_Macro.Print()

In [ ]:
NumEvts_Gen = root2array(SuperSimfile, treename='G4SettingsInfoDir/runtime', branches='Events')
NumEvts_Pro = root2array(SuperSimfile, treename='G4SimDir/g4dmcEvent',       branches='EventNum')
NumEvts_Gen = int(sum(NumEvts_Gen))
NumEvts_Pro = len(NumEvts_Pro)
print('Number of Generaged Events: {}'.format(NumEvts_Gen))
print('Number of Processed Events: {}'.format(NumEvts_Pro))

In [ ]:
# recoil energy: energy deposited in the detector
EvtNumbers  = root2array(SuperSimfile, treename='G4SimDir/g4dmcHits', branches='EventNum')
Edeposit    = root2array(SuperSimfile, treename='G4SimDir/g4dmcHits', branches='trueEnergy')
Erecoil     = Edeposit
# expected phonon energy collected
Y_lindhard  = root2array(SuperSimfile, treename="G4SimDir/g4dmcHits", branches = 'LindhardYield')
PhE_exp     = Erecoil*(1+Y_lindhard*70./2.96)/2. # divided by 2, only one side is read out in CDMSlite)
PhE_exp     = PhE_exp/1e3  # unit: eV->keV
# collected phonon energy in DMC, read out directly from g4dmcEvent
PhE_DMC     = root2array(SuperSimfile, treename='G4SimDir/g4dmcEvent', branches='PhononE')
PhE_DMC     = PhE_DMC/1e3  # unit: eV->keV
# Calculate other further quantities
PhEsum_DMC  = np.array([np.sum(i) for i in PhE_DMC])
CollectEff  = PhEsum_DMC/PhE_exp
PhE_DMC     = np.stack(PhE_DMC, axis=0)  # originally an array of arrays. Convert to a single array

In [ ]:
# Positions
PositionX = root2array(SuperSimfile, treename='G4SimDir/g4dmcHits', branches='X')
PositionY = root2array(SuperSimfile, treename='G4SimDir/g4dmcHits', branches='Y')
PositionZ = root2array(SuperSimfile, treename='G4SimDir/g4dmcHits', branches='Z')
X = PositionX*1e3    # unit: m->mm
Y = PositionY*1e3    # unit: m->mm
Z = PositionZ*1e3    # unit: m->mm
R = np.sqrt(X**2+Y**2)

In [ ]:
PhE_A   = PhE_DMC[:,0]
PhE_B   = PhE_DMC[:,1]
PhE_C   = PhE_DMC[:,2]
PhE_D   = PhE_DMC[:,3]
PhE_sum = PhEsum_DMC
PhE_BCD = PhE_B+PhE_C+PhE_D

In [ ]:
EnergyTotal = 123.24 #keV

# 1. Phonon Energy vs R

In [ ]:
cut=CollectEff>=0.98
np.count_nonzero(cut)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)
fig.suptitle('Phonon Energy in DMC vs R, ColEff≥0.98')
axs[0].plot(R[cut], PhE_BCD[cut], '.',markersize=2,label='PB+PC+PD')
#axs[0].plot(R[~cut],PhE_BCD[~cut],'.',markersize=2,label='PB+PC+PD'+', ColEff<0.98')
axs[0].legend()
axs[1].plot(R[cut], PhE_A[cut], '.',markersize=2,label='PA')
#axs[1].plot(R[~cut],PhE_A[~cut],'.',markersize=2,label='PA'+', ColEff<0.98')
axs[1].legend()

for ax in axs.flat:
    ax.set(xlabel='R [mm]',ylabel='Phonon Energy [keV]')
    ax.label_outer()   
    
plt.show()

# 2. Two Circle Model and its Prediction

## 2.1 One Circle for Fast Component

In [ ]:
r1 = 15
R1 = 31.3
R2 = 36.1
R3 = 38.1

step = 0.1
d    = np.linspace(0,R3,int(R3/step))

In [ ]:
plt.figure(figsize=(10, 8), dpi=100)
Rh = 30
circle_Det   = plt.Circle(( 0, 0), R3, edgecolor='k',fill=False)
circle_PA    = plt.Circle(( 0, 0), R2, edgecolor='k',fill=False)
circle_PBCD  = plt.Circle(( 0, 0), R1, edgecolor='k',fill=False)

plt.gca().add_patch(circle_Det)
plt.gca().add_patch(circle_PA)
plt.gca().add_patch(circle_PBCD)

plt.text(-3, 31,'PA', fontsize='30')
#plt.text( -1,  31, 'A', size='xx-large')
plt.text(-6, 20,'PBCD', fontsize='30')


plt.xlim(-40-2,40+20+2)
plt.ylim(-40+0.5,40-0.5)
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8), dpi=100)
Rh = 30
circle_Det   = plt.Circle(( 0, 0), R3, edgecolor='k',fill=False)
circle_PA    = plt.Circle(( 0, 0), R2, edgecolor='k',fill=False)
circle_PBCD  = plt.Circle(( 0, 0), R1, edgecolor='k',fill=False)
circle_slow  = plt.Circle(( 0, 0), R2, color='lightblue', alpha=0.5)
circle_fast1 = plt.Circle((Rh, 0), r1, color='orange', alpha=0.5)
#circle_fast2 = plt.Circle((Rh, 0), r2, color='orange', alpha=0.5)
#circle_fast3 = plt.Circle((Rh, 0), r3, color='orange', alpha=0.5)
plt.plot(Rh, 0,'*',color='k',markersize=30)
plt.gca().add_patch(circle_Det)
plt.gca().add_patch(circle_PA)
plt.gca().add_patch(circle_PBCD)
plt.gca().add_patch(circle_slow)
plt.gca().add_patch(circle_fast1)
#plt.gca().add_patch(circle_fast2)
#plt.gca().add_patch(circle_fast3)

plt.text(-3, 31,'PA', fontsize='30')
#plt.text( -1,  31, 'A', size='xx-large')
plt.text(-6, 20,'PBCD', fontsize='30')


plt.xlim(-40-2,40+20+2)
plt.ylim(-40+0.5,40-0.5)
plt.axis('off')
plt.show()

In [ ]:
pi = np.pi
Area_r1 = pi*r1**2
#Area_r2 = pi*r2**2
#Area_r3 = pi*r3**2
Area_R1 = pi*R1**2
Area_R2 = pi*R2**2
Area_R3 = pi*R3**2

In [ ]:
Area_PT   = pi*R2**2
Area_PA   = pi*R2**2-pi*R1**2
Area_PBCD = pi*R1**2

In [ ]:
Area_slowPBCD  = Area_PBCD
Area_slowPA    = Area_PA
Ratio_slowPBCD = Area_PBCD/Area_PT
Ratio_slowPA   = Area_PA/Area_PT

In [ ]:
EnergyfastRatio1 = 6.00/100
#EnergyfastRatio2 = 3.00/100
#EnergyfastRatio3 = 2.00/100
EnergyslowRatio  = 1-EnergyfastRatio1

### 2.1.1 Fraction of Phonon Energy BCD

In [ ]:
step = 0.1
## overlap r1R1
d1    = np.linspace(R1-r1+step,R3,int((R3-R1+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R1**2)/(2*d1*r1))
term2 = (R1**2)*np.arccos((d1**2-r1**2+R1**2)/(2*d1*R1))
term3 = -0.5*np.sqrt((d1+r1-R1)*(d1-r1+R1)*(-d1+r1+R1)*(d1+r1+R1))
Area_overlap_r1R1  = term1+term2+term3
Area_lune_r1R1     = Area_r1-Area_overlap_r1R1
Ratio_overlap_r1R1 = Area_overlap_r1R1/Area_r1
Ratio_lune_r1R1    = 1-Ratio_overlap_r1R1
Ratio_r1inBCD = np.concatenate((np.ones(len(d)-len(d1)),Ratio_overlap_r1R1),axis=None)

In [ ]:
Energy_BCD = EnergyTotal*( EnergyfastRatio1*Ratio_r1inBCD
                          +EnergyslowRatio*Ratio_slowPBCD)

In [ ]:
plt.plot(R[cut], PhE_BCD[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_BCD/EnergyTotal,label='2CirModel')

plt.title('Phonon Energy Ratio, PBCD'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.1.2 Fraction of Phonon Energy A

In [ ]:
step = 0.1
## overlap r1R2
d1    = np.linspace(R2-r1+step,R3,int((R3-R2+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R2**2)/(2*d1*r1))
term2 = (R2**2)*np.arccos((d1**2-r1**2+R2**2)/(2*d1*R2))
term3 = -0.5*np.sqrt((d1+r1-R2)*(d1-r1+R2)*(-d1+r1+R2)*(d1+r1+R2))
Area_overlap_r1R2  = term1+term2+term3
Area_lune_r1R2     = Area_r1-Area_overlap_r1R2
Ratio_overlap_r1R2 = Area_overlap_r1R2/Area_r1
Ratio_lune_r1R2    = 1-Ratio_overlap_r1R2
Ratio_r1inA = 1-Ratio_r1inBCD#-np.concatenate((np.zeros(len(d)-len(d1)),Ratio_lune_r1R2),axis=None)

In [ ]:
Energy_A = EnergyTotal*( EnergyfastRatio1*Ratio_r1inA
                        +EnergyslowRatio*Ratio_slowPA)

In [ ]:
plt.plot(R[cut], PhE_A[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/EnergyTotal,label='2CirModel')

plt.title('Phonon Energy Ratio, PA'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.2.3 Phonon Energy A2BCD

In [ ]:
PhE_A2BCD = PhE_A/PhE_BCD
plt.plot(R[cut], PhE_A2BCD[cut], '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/Energy_BCD,label='2CirModel')
#plt.plot(R[~cut],PhE_A2BCD[~cut],'.',label='ColEff<0.98')
#plt.hlines(0.35,0,38.1,linestyles='dashed',colors='g',lw=2,label='A/BCD=0.35')
plt.title('PhE_A/PhE_(B+C+D) vs R'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('PhE_A/PhE_(B+C+D)')
plt.legend()
plt.show()

## 2.2 Two Circles for Fast Component

In [ ]:
r1 = 7
r2 = 15
#r3 = 25
R1 = 31.3
R2 = 36.1
R3 = 38.1

step = 0.1
d    = np.linspace(0,R3,int(R3/step))

In [ ]:
plt.figure(figsize=(10, 8), dpi=100)
Rh = 20
circle_Det   = plt.Circle(( 0, 0), R3, edgecolor='k',fill=False)
circle_PA    = plt.Circle(( 0, 0), R2, edgecolor='k',fill=False)
circle_PBCD  = plt.Circle(( 0, 0), R1, edgecolor='k',fill=False)
circle_slow  = plt.Circle(( 0, 0), R2, color='lightblue', alpha=0.5)
circle_fast1 = plt.Circle((Rh, 0), r1, color='orange', alpha=0.5)
circle_fast2 = plt.Circle((Rh, 0), r2, color='orange', alpha=0.5)
#circle_fast3 = plt.Circle((Rh, 0), r3, color='orange', alpha=0.5)
plt.plot(Rh, 0,'*',color='k',markersize=30)
plt.gca().add_patch(circle_Det)
plt.gca().add_patch(circle_PA)
plt.gca().add_patch(circle_PBCD)
plt.gca().add_patch(circle_slow)
plt.gca().add_patch(circle_fast1)
plt.gca().add_patch(circle_fast2)
#plt.gca().add_patch(circle_fast3)

plt.text(-3, 31,'PA', fontsize='30')
#plt.text( -1,  31, 'A', size='xx-large')
plt.text(-6, 20,'PBCD', fontsize='30')


plt.xlim(-40-2,40+20+2)
plt.ylim(-40+0.5,40-0.5)
plt.axis('off')
plt.show()

In [ ]:
EnergyfastRatio1 = 3.00/100
EnergyfastRatio2 = 3.00/100
#EnergyfastRatio3 = 2.00/100
EnergyslowRatio  = 1-EnergyfastRatio1-EnergyfastRatio2

In [ ]:
pi = np.pi
Area_r1 = pi*r1**2
Area_r2 = pi*r2**2
#Area_r3 = pi*r3**2
Area_R1 = pi*R1**2
Area_R2 = pi*R2**2
Area_R3 = pi*R3**2

In [ ]:
Area_PT   = pi*R2**2
Area_PA   = pi*R2**2-pi*R1**2
Area_PBCD = pi*R1**2

In [ ]:
Area_slowPBCD  = Area_PBCD
Area_slowPA    = Area_PA
Ratio_slowPBCD = Area_PBCD/Area_PT
Ratio_slowPA   = Area_PA/Area_PT

### 2.2.1 Fraction of Phonon Energy BCD

In [ ]:
step = 0.1
## overlap r1R1
d1    = np.linspace(R1-r1+step,R3,int((R3-R1+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R1**2)/(2*d1*r1))
term2 = (R1**2)*np.arccos((d1**2-r1**2+R1**2)/(2*d1*R1))
term3 = -0.5*np.sqrt((d1+r1-R1)*(d1-r1+R1)*(-d1+r1+R1)*(d1+r1+R1))
Area_overlap_r1R1  = term1+term2+term3
Area_lune_r1R1     = Area_r1-Area_overlap_r1R1
Ratio_overlap_r1R1 = Area_overlap_r1R1/Area_r1
Ratio_lune_r1R1    = 1-Ratio_overlap_r1R1
Ratio_r1inBCD = np.concatenate((np.ones(len(d)-len(d1)),Ratio_overlap_r1R1),axis=None)

In [ ]:
## overlap r2R1
d2    = np.linspace(R1-r2+step,R3,int((R3-R1+r2)/step))
term1 = (r2**2)*np.arccos((d2**2+r2**2-R1**2)/(2*d2*r2))
term2 = (R1**2)*np.arccos((d2**2-r2**2+R1**2)/(2*d2*R1))
term3 = -0.5*np.sqrt((d2+r2-R1)*(d2-r2+R1)*(-d2+r2+R1)*(d2+r2+R1))
Area_overlap_r2R1  = term1+term2+term3
Area_lune_r2R1     = Area_r2-Area_overlap_r2R1
Ratio_overlap_r2R1 = Area_overlap_r2R1/Area_r2
Ratio_lune_r2R1    = 1-Ratio_overlap_r2R1
Ratio_r2inBCD = np.concatenate((np.ones(len(d)-len(d2)),Ratio_overlap_r2R1),axis=None)

In [ ]:
Energy_BCD = EnergyTotal*( EnergyfastRatio1*Ratio_r1inBCD
                          +EnergyfastRatio2*Ratio_r2inBCD
                          +EnergyslowRatio*Ratio_slowPBCD)

In [ ]:
plt.plot(R[cut], PhE_BCD[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_BCD/EnergyTotal,label='2CirModel')

plt.title('Phonon Energy Ratio, PBCD'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.2.2 Fraction of Phonon Energy A

In [ ]:
step = 0.1
## overlap r1R2
d1    = np.linspace(R2-r1+step,R3,int((R3-R2+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R2**2)/(2*d1*r1))
term2 = (R2**2)*np.arccos((d1**2-r1**2+R2**2)/(2*d1*R2))
term3 = -0.5*np.sqrt((d1+r1-R2)*(d1-r1+R2)*(-d1+r1+R2)*(d1+r1+R2))
Area_overlap_r1R2  = term1+term2+term3
Area_lune_r1R2     = Area_r1-Area_overlap_r1R2
Ratio_overlap_r1R2 = Area_overlap_r1R2/Area_r1
Ratio_lune_r1R2    = 1-Ratio_overlap_r1R2
Ratio_r1inA = 1-Ratio_r1inBCD#-np.concatenate((np.zeros(len(d)-len(d1)),Ratio_lune_r1R2),axis=None)

In [ ]:
step = 0.1
## overlap r2R2
d2    = np.linspace(R2-r2+step,R3,int((R3-R2+r2)/step))
term1 = (r2**2)*np.arccos((d2**2+r2**2-R2**2)/(2*d2*r2))
term2 = (R2**2)*np.arccos((d2**2-r2**2+R2**2)/(2*d2*R2))
term3 = -0.5*np.sqrt((d2+r2-R2)*(d2-r2+R2)*(-d2+r2+R2)*(d2+r2+R2))
Area_overlap_r2R2  = term1+term2+term3
Area_lune_r2R2     = Area_r2-Area_overlap_r2R2
Ratio_overlap_r2R2 = Area_overlap_r2R2/Area_r2
Ratio_lune_r2R2    = 1-Ratio_overlap_r2R2
Ratio_r2inA = 1-Ratio_r2inBCD#-np.concatenate((np.zeros(len(d)-len(d2)),Ratio_lune_r2R2),axis=None)

In [ ]:
Energy_A = EnergyTotal*( EnergyfastRatio1*Ratio_r1inA
                        +EnergyfastRatio2*Ratio_r2inA
                        +EnergyslowRatio*Ratio_slowPA)

In [ ]:
plt.plot(R[cut], PhE_A[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/EnergyTotal,label='2CirModel')

plt.title('Phonon Energy Ratio, PA'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.2.3 Phonon Energy A2BCD

In [ ]:
PhE_A2BCD = PhE_A/PhE_BCD
plt.plot(R[cut], PhE_A2BCD[cut], '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/Energy_BCD,label='2CirModel')
#plt.plot(R[~cut],PhE_A2BCD[~cut],'.',label='ColEff<0.98')
#plt.hlines(0.35,0,38.1,linestyles='dashed',colors='g',lw=2,label='A/BCD=0.35')
plt.title('PhE_A/PhE_(B+C+D) vs R'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('PhE_A/PhE_(B+C+D)')
plt.legend()
plt.show()

## 2.3 Three Circles for Fast Component

In [ ]:
r1 = 7
r2 = 15
r3 = 25
R1 = 31.3
R2 = 36.1
R3 = 38.1

step = 0.1
d    = np.linspace(0,R3,int(R3/step))

In [ ]:
plt.figure(figsize=(10, 8), dpi=100)
Rh = 20
circle_Det   = plt.Circle(( 0, 0), R3, edgecolor='k',fill=False)
circle_PA    = plt.Circle(( 0, 0), R2, edgecolor='k',fill=False)
circle_PBCD  = plt.Circle(( 0, 0), R1, edgecolor='k',fill=False)
circle_slow  = plt.Circle(( 0, 0), R2, color='lightblue', alpha=0.5)
circle_fast1 = plt.Circle((Rh, 0), r1, color='orange', alpha=0.5)
circle_fast2 = plt.Circle((Rh, 0), r2, color='orange', alpha=0.5)
circle_fast3 = plt.Circle((Rh, 0), r3, color='orange', alpha=0.5)
plt.plot(Rh, 0,'*',color='k',markersize=30)
plt.gca().add_patch(circle_Det)
plt.gca().add_patch(circle_PA)
plt.gca().add_patch(circle_PBCD)
plt.gca().add_patch(circle_slow)
plt.gca().add_patch(circle_fast1)
plt.gca().add_patch(circle_fast2)
plt.gca().add_patch(circle_fast3)

plt.text(-3, 31,'PA', fontsize='30')
#plt.text( -1,  31, 'A', size='xx-large')
plt.text(-6, 20,'PBCD', fontsize='30')


plt.xlim(-40-2,40+20+2)
plt.ylim(-40+0.5,40-0.5)
plt.axis('off')
plt.show()

In [ ]:
EnergyfastRatio1 = 2.00/100
EnergyfastRatio2 = 2.00/100
EnergyfastRatio3 = 2.00/100
EnergyslowRatio  = 1-EnergyfastRatio1-EnergyfastRatio2-EnergyfastRatio3

In [ ]:
pi = np.pi
Area_r1 = pi*r1**2
Area_r2 = pi*r2**2
Area_r3 = pi*r3**2
Area_R1 = pi*R1**2
Area_R2 = pi*R2**2
Area_R3 = pi*R3**2

In [ ]:
Area_PT   = pi*R2**2
Area_PA   = pi*R2**2-pi*R1**2
Area_PBCD = pi*R1**2

In [ ]:
Area_slowPBCD  = Area_PBCD
Area_slowPA    = Area_PA
Ratio_slowPBCD = Area_PBCD/Area_PT
Ratio_slowPA   = Area_PA/Area_PT

### 2.3.1 Fraction of Phonon Energy BCD

In [ ]:
step = 0.1
## overlap r1R1
d1    = np.linspace(R1-r1+step,R3,int((R3-R1+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R1**2)/(2*d1*r1))
term2 = (R1**2)*np.arccos((d1**2-r1**2+R1**2)/(2*d1*R1))
term3 = -0.5*np.sqrt((d1+r1-R1)*(d1-r1+R1)*(-d1+r1+R1)*(d1+r1+R1))
Area_overlap_r1R1  = term1+term2+term3
Area_lune_r1R1     = Area_r1-Area_overlap_r1R1
Ratio_overlap_r1R1 = Area_overlap_r1R1/Area_r1
Ratio_lune_r1R1    = 1-Ratio_overlap_r1R1
Ratio_r1inBCD = np.concatenate((np.ones(len(d)-len(d1)),Ratio_overlap_r1R1),axis=None)

In [ ]:
## overlap r2R1
d2    = np.linspace(R1-r2+step,R3,int((R3-R1+r2)/step))
term1 = (r2**2)*np.arccos((d2**2+r2**2-R1**2)/(2*d2*r2))
term2 = (R1**2)*np.arccos((d2**2-r2**2+R1**2)/(2*d2*R1))
term3 = -0.5*np.sqrt((d2+r2-R1)*(d2-r2+R1)*(-d2+r2+R1)*(d2+r2+R1))
Area_overlap_r2R1  = term1+term2+term3
Area_lune_r2R1     = Area_r2-Area_overlap_r2R1
Ratio_overlap_r2R1 = Area_overlap_r2R1/Area_r2
Ratio_lune_r2R1    = 1-Ratio_overlap_r2R1
Ratio_r2inBCD = np.concatenate((np.ones(len(d)-len(d2)),Ratio_overlap_r2R1),axis=None)

In [ ]:
## overlap r3R1
d3    = np.linspace(R1-r3+step,R3,int((R3-R1+r3)/step))
term1 = (r3**2)*np.arccos((d3**2+r3**2-R1**2)/(2*d3*r3))
term2 = (R1**2)*np.arccos((d3**2-r3**2+R1**2)/(2*d3*R1))
term3 = -0.5*np.sqrt((d3+r3-R1)*(d3-r3+R1)*(-d3+r3+R1)*(d3+r3+R1))
Area_overlap_r3R1  = term1+term2+term3
Area_lune_r3R1     = Area_r3-Area_overlap_r3R1
Ratio_overlap_r3R1 = Area_overlap_r3R1/Area_r3
Ratio_lune_r3R1    = 1-Ratio_overlap_r3R1
Ratio_r3inBCD = np.concatenate((np.ones(len(d)-len(d3)),Ratio_overlap_r3R1),axis=None)

In [ ]:
Energy_BCD = EnergyTotal*( EnergyfastRatio1*Ratio_r1inBCD
                          +EnergyfastRatio2*Ratio_r2inBCD
                          +EnergyfastRatio3*Ratio_r3inBCD
                          +EnergyslowRatio*Ratio_slowPBCD)

In [ ]:
plt.plot(R[cut], PhE_BCD[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_BCD/EnergyTotal,label='2CirModel')

plt.title('Phonon Energy Ratio, PBCD'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.3.2 Fraction of Phonon Energy A

In [ ]:
step = 0.1
## overlap r1R2
d1    = np.linspace(R2-r1+step,R3,int((R3-R2+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R2**2)/(2*d1*r1))
term2 = (R2**2)*np.arccos((d1**2-r1**2+R2**2)/(2*d1*R2))
term3 = -0.5*np.sqrt((d1+r1-R2)*(d1-r1+R2)*(-d1+r1+R2)*(d1+r1+R2))
Area_overlap_r1R2  = term1+term2+term3
Area_lune_r1R2     = Area_r1-Area_overlap_r1R2
Ratio_overlap_r1R2 = Area_overlap_r1R2/Area_r1
Ratio_lune_r1R2    = 1-Ratio_overlap_r1R2
Ratio_r1inA = 1-Ratio_r1inBCD#-np.concatenate((np.zeros(len(d)-len(d1)),Ratio_lune_r1R2),axis=None)

In [ ]:
step = 0.1
## overlap r2R2
d2    = np.linspace(R2-r2+step,R3,int((R3-R2+r2)/step))
term1 = (r2**2)*np.arccos((d2**2+r2**2-R2**2)/(2*d2*r2))
term2 = (R2**2)*np.arccos((d2**2-r2**2+R2**2)/(2*d2*R2))
term3 = -0.5*np.sqrt((d2+r2-R2)*(d2-r2+R2)*(-d2+r2+R2)*(d2+r2+R2))
Area_overlap_r2R2  = term1+term2+term3
Area_lune_r2R2     = Area_r2-Area_overlap_r2R2
Ratio_overlap_r2R2 = Area_overlap_r2R2/Area_r2
Ratio_lune_r2R2    = 1-Ratio_overlap_r2R2
Ratio_r2inA = 1-Ratio_r2inBCD#-np.concatenate((np.zeros(len(d)-len(d2)),Ratio_lune_r2R2),axis=None)

In [ ]:
step = 0.1
## overlap r3R2
d3    = np.linspace(R2-r3+step,R3,int((R3-R2+r3)/step))
term1 = (r3**2)*np.arccos((d3**2+r3**2-R2**2)/(2*d3*r3))
term2 = (R2**2)*np.arccos((d3**2-r3**2+R2**2)/(2*d3*R2))
term3 = -0.5*np.sqrt((d3+r3-R2)*(d3-r3+R2)*(-d3+r3+R2)*(d3+r3+R2))
Area_overlap_r3R2  = term1+term2+term3
Area_lune_r3R2     = Area_r3-Area_overlap_r3R2
Ratio_overlap_r3R2 = Area_overlap_r3R2/Area_r3
Ratio_lune_r3R2    = 1-Ratio_overlap_r3R2
Ratio_r3inA = 1-Ratio_r3inBCD#-np.concatenate((np.zeros(len(d)-len(d3)),Ratio_lune_r3R2),axis=None)

In [ ]:
Energy_A = EnergyTotal*( EnergyfastRatio1*Ratio_r1inA
                        +EnergyfastRatio2*Ratio_r2inA
                        +EnergyfastRatio3*Ratio_r3inA
                        +EnergyslowRatio*Ratio_slowPA)

In [ ]:
plt.plot(R[cut], PhE_A[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/EnergyTotal,label='2CirModel')

plt.title('Phonon Energy Ratio, PA'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.3.3 Phonon Energy A2BCD

In [ ]:
PhE_A2BCD = PhE_A/PhE_BCD
plt.plot(R[cut], PhE_A2BCD[cut], '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/Energy_BCD,label='2CirModel')
#plt.plot(R[~cut],PhE_A2BCD[~cut],'.',label='ColEff<0.98')
#plt.hlines(0.35,0,38.1,linestyles='dashed',colors='g',lw=2,label='A/BCD=0.35')
plt.title('PhE_A/PhE_(B+C+D) vs R'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('PhE_A/PhE_(B+C+D)')
plt.legend()
plt.show()

## 2.4 Four Circles for Fast Component

In [ ]:
r1 = 7
r2 = 15
r3 = 22
r4 = 29.9
R1 = 31.3
R2 = 36.1
R3 = 38.1

step = 0.1
d    = np.linspace(0,R3,int(R3/step))

In [ ]:
plt.figure(figsize=(10, 8), dpi=100)
Rh = 20
circle_Det   = plt.Circle(( 0, 0), R3, edgecolor='k',fill=False)
circle_PA    = plt.Circle(( 0, 0), R2, edgecolor='k',fill=False)
circle_PBCD  = plt.Circle(( 0, 0), R1, edgecolor='k',fill=False)
circle_slow  = plt.Circle(( 0, 0), R2, color='lightblue', alpha=0.5)
circle_fast1 = plt.Circle((Rh, 0), r1, color='orange', alpha=0.5)
circle_fast2 = plt.Circle((Rh, 0), r2, color='orange', alpha=0.5)
circle_fast3 = plt.Circle((Rh, 0), r3, color='orange', alpha=0.5)
circle_fast4 = plt.Circle((Rh, 0), r4, color='orange', alpha=0.5)
plt.plot(Rh, 0,'*',color='k',markersize=30)
plt.gca().add_patch(circle_Det)
plt.gca().add_patch(circle_PA)
plt.gca().add_patch(circle_PBCD)
plt.gca().add_patch(circle_slow)
plt.gca().add_patch(circle_fast1)
plt.gca().add_patch(circle_fast2)
plt.gca().add_patch(circle_fast3)
plt.gca().add_patch(circle_fast4)

plt.text(-3, 31,'PA', fontsize='30')
#plt.text( -1,  31, 'A', size='xx-large')
plt.text(-6, 20,'PBCD', fontsize='30')


plt.xlim(-40-2,40+20+2)
plt.ylim(-40+0.5,40-0.5)
plt.axis('off')
plt.show()

In [ ]:
EnergyfastRatio1 = 1.50/100
EnergyfastRatio2 = 1.50/100
EnergyfastRatio3 = 1.50/100
EnergyfastRatio4 = 1.50/100
EnergyslowRatio  = 1-EnergyfastRatio1-EnergyfastRatio2-EnergyfastRatio3-EnergyfastRatio4

In [ ]:
pi = np.pi
Area_r1 = pi*r1**2
Area_r2 = pi*r2**2
Area_r3 = pi*r3**2
Area_r4 = pi*r4**2
Area_R1 = pi*R1**2
Area_R2 = pi*R2**2
Area_R3 = pi*R3**2

In [ ]:
Area_PT   = pi*R2**2
Area_PA   = pi*R2**2-pi*R1**2
Area_PBCD = pi*R1**2

In [ ]:
Area_slowPBCD  = Area_PBCD
Area_slowPA    = Area_PA
Ratio_slowPBCD = Area_PBCD/Area_PT
Ratio_slowPA   = Area_PA/Area_PT

### 2.4.1 Fraction of Phonon Energy BCD

In [ ]:
step = 0.1
## overlap r1R1
d1    = np.linspace(R1-r1+step,R3,int((R3-R1+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R1**2)/(2*d1*r1))
term2 = (R1**2)*np.arccos((d1**2-r1**2+R1**2)/(2*d1*R1))
term3 = -0.5*np.sqrt((d1+r1-R1)*(d1-r1+R1)*(-d1+r1+R1)*(d1+r1+R1))
Area_overlap_r1R1  = term1+term2+term3
Area_lune_r1R1     = Area_r1-Area_overlap_r1R1
Ratio_overlap_r1R1 = Area_overlap_r1R1/Area_r1
Ratio_lune_r1R1    = 1-Ratio_overlap_r1R1
Ratio_r1inBCD = np.concatenate((np.ones(len(d)-len(d1)),Ratio_overlap_r1R1),axis=None)

In [ ]:
## overlap r2R1
d2    = np.linspace(R1-r2+step,R3,int((R3-R1+r2)/step))
term1 = (r2**2)*np.arccos((d2**2+r2**2-R1**2)/(2*d2*r2))
term2 = (R1**2)*np.arccos((d2**2-r2**2+R1**2)/(2*d2*R1))
term3 = -0.5*np.sqrt((d2+r2-R1)*(d2-r2+R1)*(-d2+r2+R1)*(d2+r2+R1))
Area_overlap_r2R1  = term1+term2+term3
Area_lune_r2R1     = Area_r2-Area_overlap_r2R1
Ratio_overlap_r2R1 = Area_overlap_r2R1/Area_r2
Ratio_lune_r2R1    = 1-Ratio_overlap_r2R1
Ratio_r2inBCD = np.concatenate((np.ones(len(d)-len(d2)),Ratio_overlap_r2R1),axis=None)

In [ ]:
## overlap r3R1
d3    = np.linspace(R1-r3+step,R3,int((R3-R1+r3)/step))
term1 = (r3**2)*np.arccos((d3**2+r3**2-R1**2)/(2*d3*r3))
term2 = (R1**2)*np.arccos((d3**2-r3**2+R1**2)/(2*d3*R1))
term3 = -0.5*np.sqrt((d3+r3-R1)*(d3-r3+R1)*(-d3+r3+R1)*(d3+r3+R1))
Area_overlap_r3R1  = term1+term2+term3
Area_lune_r3R1     = Area_r3-Area_overlap_r3R1
Ratio_overlap_r3R1 = Area_overlap_r3R1/Area_r3
Ratio_lune_r3R1    = 1-Ratio_overlap_r3R1
Ratio_r3inBCD = np.concatenate((np.ones(len(d)-len(d3)),Ratio_overlap_r3R1),axis=None)

In [ ]:
d4    = np.linspace(R1-r4+step,R3,int((R3-R1+r4)/step))
term1 = (r4**2)*np.arccos((d4**2+r4**2-R1**2)/(2*d4*r4))
term2 = (R1**2)*np.arccos((d4**2-r4**2+R1**2)/(2*d4*R1))
term3 = -0.5*np.sqrt((d4+r4-R1)*(d4-r4+R1)*(-d4+r4+R1)*(d4+r4+R1))
Area_overlap_r4R1  = term1+term2+term3
Area_lune_r4R1     = Area_r4-Area_overlap_r4R1
Ratio_overlap_r4R1 = Area_overlap_r4R1/Area_r4
Ratio_lune_r4R1    = 1-Ratio_overlap_r4R1
Ratio_r4inBCD = np.concatenate((np.ones(len(d)-len(d4)),Ratio_overlap_r4R1),axis=None)

In [ ]:
Energy_BCD = EnergyTotal*( EnergyfastRatio1*Ratio_r1inBCD
                          +EnergyfastRatio2*Ratio_r2inBCD
                          +EnergyfastRatio3*Ratio_r3inBCD
                          +EnergyfastRatio3*Ratio_r4inBCD
                          +EnergyslowRatio*Ratio_slowPBCD)

In [ ]:
plt.plot(R[cut], PhE_BCD[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_BCD/EnergyTotal,label='2CirModel')
plt.title('Phonon Energy Ratio, PBCD'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.legend()
plt.show()

### 2.3.2 Fraction of Phonon Energy A

In [ ]:
step = 0.1
## overlap r1R2
d1    = np.linspace(R2-r1+step,R3,int((R3-R2+r1)/step))
term1 = (r1**2)*np.arccos((d1**2+r1**2-R2**2)/(2*d1*r1))
term2 = (R2**2)*np.arccos((d1**2-r1**2+R2**2)/(2*d1*R2))
term3 = -0.5*np.sqrt((d1+r1-R2)*(d1-r1+R2)*(-d1+r1+R2)*(d1+r1+R2))
Area_overlap_r1R2  = term1+term2+term3
Area_lune_r1R2     = Area_r1-Area_overlap_r1R2
Ratio_overlap_r1R2 = Area_overlap_r1R2/Area_r1
Ratio_lune_r1R2    = 1-Ratio_overlap_r1R2
Ratio_r1inA = 1-Ratio_r1inBCD#-np.concatenate((np.zeros(len(d)-len(d1)),Ratio_lune_r1R2),axis=None)

In [ ]:
step = 0.1
## overlap r2R2
d2    = np.linspace(R2-r2+step,R3,int((R3-R2+r2)/step))
term1 = (r2**2)*np.arccos((d2**2+r2**2-R2**2)/(2*d2*r2))
term2 = (R2**2)*np.arccos((d2**2-r2**2+R2**2)/(2*d2*R2))
term3 = -0.5*np.sqrt((d2+r2-R2)*(d2-r2+R2)*(-d2+r2+R2)*(d2+r2+R2))
Area_overlap_r2R2  = term1+term2+term3
Area_lune_r2R2     = Area_r2-Area_overlap_r2R2
Ratio_overlap_r2R2 = Area_overlap_r2R2/Area_r2
Ratio_lune_r2R2    = 1-Ratio_overlap_r2R2
Ratio_r2inA = 1-Ratio_r2inBCD#-np.concatenate((np.zeros(len(d)-len(d2)),Ratio_lune_r2R2),axis=None)

In [ ]:
step = 0.1
## overlap r3R2
d3    = np.linspace(R2-r3+step,R3,int((R3-R2+r3)/step))
term1 = (r3**2)*np.arccos((d3**2+r3**2-R2**2)/(2*d3*r3))
term2 = (R2**2)*np.arccos((d3**2-r3**2+R2**2)/(2*d3*R2))
term3 = -0.5*np.sqrt((d3+r3-R2)*(d3-r3+R2)*(-d3+r3+R2)*(d3+r3+R2))
Area_overlap_r3R2  = term1+term2+term3
Area_lune_r3R2     = Area_r3-Area_overlap_r3R2
Ratio_overlap_r3R2 = Area_overlap_r3R2/Area_r3
Ratio_lune_r3R2    = 1-Ratio_overlap_r3R2
Ratio_r3inA = 1-Ratio_r3inBCD#-np.concatenate((np.zeros(len(d)-len(d3)),Ratio_lune_r3R2),axis=None)

In [ ]:
step = 0.1
## overlap r4R2
d4    = np.linspace(R2-r4+step,R3,int((R3-R2+r4)/step))
term1 = (r4**2)*np.arccos((d4**2+r4**2-R2**2)/(2*d4*r4))
term2 = (R2**2)*np.arccos((d4**2-r4**2+R2**2)/(2*d4*R2))
term3 = -0.5*np.sqrt((d4+r4-R2)*(d4-r4+R2)*(-d4+r4+R2)*(d4+r4+R2))
Area_overlap_r4R2  = term1+term2+term3
Area_lune_r4R2     = Area_r4-Area_overlap_r4R2
Ratio_overlap_r4R2 = Area_overlap_r4R2/Area_r4
Ratio_lune_r4R2    = 1-Ratio_overlap_r4R2
Ratio_r4inA = 1-Ratio_r4inBCD#-np.concatenate((np.zeros(len(d)-len(d4)),Ratio_lune_r4R2),axis=None)

In [ ]:
Energy_A = EnergyTotal*( EnergyfastRatio1*Ratio_r1inA
                        +EnergyfastRatio2*Ratio_r2inA
                        +EnergyfastRatio3*Ratio_r3inA
                        +EnergyfastRatio4*Ratio_r4inA
                        +EnergyslowRatio*Ratio_slowPA)

In [ ]:
plt.plot(R[cut], PhE_A[cut]/EnergyTotal, '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/EnergyTotal,label='2CirModel')
plt.title('Phonon Energy Ratio, PA'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('Phonon Energy Ratio')
plt.show()

### 2.3.3 Phonon Energy A2BCD

In [ ]:
PhE_A2BCD = PhE_A/PhE_BCD
plt.plot(R[cut], PhE_A2BCD[cut], '.',markersize=2,label='SimData')
plt.plot(d,Energy_A/Energy_BCD,label='2CirModel')
#plt.plot(R[~cut],PhE_A2BCD[~cut],'.',label='ColEff<0.98')
#plt.hlines(0.35,0,38.1,linestyles='dashed',colors='g',lw=2,label='A/BCD=0.35')
plt.title('PhE_A/PhE_(B+C+D) vs R'+'\nSimData and 2CirModel')
plt.xlabel('R [mm]')
plt.ylabel('PhE_A/PhE_(B+C+D)')
plt.legend()
plt.show()